# 🚀 Mode Démo Extract_Depot_Sh via Jupyter

Ce notebook exécute le mode démo du script MCP dans Jupyter (contourne les restrictions d'entreprise)

**✅ Avantage :** Fonctionne même si Python n'est pas accessible en ligne de commande


In [1]:
# 🔧 Configuration et imports (sans MCP pour éviter les restrictions d'entreprise)
import os
import shutil
import json
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Any, Optional

print("✅ Imports de base réussis")
print(f"📁 Répertoire actuel: {os.getcwd()}")

# Configuration identique au script MCP
@dataclass
class ExtractConfig:
    """Configuration pour l'extraction de fichiers"""
    base_dir: str = os.path.abspath(r"C:\Users\jcbouquin\AI\GenAI\company_process")
    source_path: str = ""
    depot_path: str = ""
    days_limit: int = 365  # Fichiers de moins d'un an
    size_threshold_kb: int = 500  # Taille minimum en KB
    extensions: List[str] = None
    
    def __post_init__(self):
        if not self.source_path:
            self.source_path = os.path.join(self.base_dir, "ProcessEx")
        if not self.depot_path:
            self.depot_path = os.path.join(self.base_dir, "Depots")
        if self.extensions is None:
            self.extensions = [".ppt", ".pptx"]

# Initialisation
config = ExtractConfig()

print(f"📋 Configuration:")
print(f"   Source: {config.source_path}")
print(f"   Depot: {config.depot_path}")
print(f"   Seuil: {config.size_threshold_kb} KB")


✅ Imports de base réussis
📁 Répertoire actuel: c:\Users\jcbouquin\AI\GenAI\company_process
📋 Configuration:
   Source: C:\Users\jcbouquin\AI\GenAI\company_process\ProcessEx
   Depot: C:\Users\jcbouquin\AI\GenAI\company_process\Depots
   Seuil: 500 KB


In [5]:
# 🔍 Fonction de recherche et filtrage (copiée du script MCP)
def search_and_filter_files(
    source_path: str = None,
    size_threshold_kb: int = None,
    days_limit: int = None,
    extensions: List[str] = None,
    show_rejected: bool = False
) -> Dict[str, Any]:
    """Recherche et filtre les fichiers selon les critères spécifiés"""
    
             # Utiliser la config par défaut si paramètres non fournis (correction du bug)
    source_path = source_path if source_path is not None else config.source_path
    size_threshold_kb = size_threshold_kb if size_threshold_kb is not None else config.size_threshold_kb
    days_limit = days_limit if days_limit is not None else config.days_limit
    extensions = extensions if extensions is not None else config.extensions
    
    # Conversion des paramètres
    size_threshold_bytes = size_threshold_kb * 1024
    date_limit = datetime.now() - timedelta(days=days_limit)
    
    if not os.path.exists(source_path):
        return {
            'success': False,
            'error': f"Chemin source introuvable: {source_path}",
            'acceptes': [],
            'rejetes': [],
            'stats': {}
        }
    
    print(f"🔍 RECHERCHE ET FILTRAGE...")
    print(f"📁 Source: {source_path}")
    print(f"📏 Taille minimum: {size_threshold_kb} KB")
    print(f"📅 Date limite: {date_limit.strftime('%d/%m/%Y')}")
    print(f"📋 Extensions: {', '.join(extensions)}")
    print("-" * 50)
    
    fichiers_acceptes = []
    fichiers_rejetes = []
    stats = {
        "total_examines": 0,
        "rejetes_extension": 0,
        "rejetes_taille": 0,
        "rejetes_date": 0,
        "rejetes_acces": 0,
        "acceptes": 0,
    }
    dossiers_stats = {}
    
    # Parcours des dossiers
    for item in os.listdir(source_path):
        item_path = os.path.join(source_path, item)
        if not os.path.isdir(item_path):
            continue
            
        print(f"📂 Analyse: {item}")
        count_dossier = 0
        
        for root, dirs, files in os.walk(item_path):
            for file in files:
                file_path = os.path.join(root, file)
                stats["total_examines"] += 1
                
                # Vérification extension
                if not any(file.lower().endswith(ext) for ext in extensions):
                    stats["rejetes_extension"] += 1
                    fichiers_rejetes.append({
                        "chemin": file_path,
                        "nom": file,
                        "raison": "Extension non supportée",
                        "dossier": item
                    })
                    continue
                
                try:
                    # Métadonnées fichier
                    taille = os.path.getsize(file_path)
                    date_mod = datetime.fromtimestamp(os.path.getmtime(file_path))
                    
                    # Filtrage par taille
                    if taille < size_threshold_bytes:
                        stats["rejetes_taille"] += 1
                        fichiers_rejetes.append({
                            "chemin": file_path,
                            "nom": file,
                            "raison": f"Taille insuffisante ({round(taille/1024, 1)} KB < {size_threshold_kb} KB)",
                            "taille": taille,
                            "date_modification": date_mod,
                            "dossier": item
                        })
                        continue
                    
                    # Fichier accepté
                    fichier_info = {
                        "chemin": file_path,
                        "nom": file,
                        "taille": taille,
                        "date_modification": date_mod,
                        "taille_mb": round(taille / (1024 * 1024), 2),
                        "taille_kb": round(taille / 1024, 1),
                        "age_jours": (datetime.now() - date_mod).days,
                        "dossier": item,
                        "classification_ok": True
                    }
                    
                    fichiers_acceptes.append(fichier_info)
                    count_dossier += 1
                    stats["acceptes"] += 1
                    
                except (OSError, PermissionError) as e:
                    stats["rejetes_acces"] += 1
                    fichiers_rejetes.append({
                        "chemin": file_path,
                        "nom": file,
                        "raison": f"Erreur d'accès: {str(e)}",
                        "dossier": item
                    })
        
        dossiers_stats[item] = count_dossier
        print(f"   ✅ {count_dossier} fichier(s) retenu(s)")
    
    # Affichage des statistiques
    print(f"\\n📊 STATISTIQUES:")
    print(f"   📋 Examinés: {stats['total_examines']}")
    print(f"   ❌ Rejetés - extension: {stats['rejetes_extension']}")
    print(f"   ❌ Rejetés - taille: {stats['rejetes_taille']}")
    print(f"   ❌ Rejetés - date: {stats['rejetes_date']}")
    print(f"   ❌ Rejetés - accès: {stats['rejetes_acces']}")
    print(f"   ✅ ACCEPTÉS: {stats['acceptes']}")
    
    if show_rejected and fichiers_rejetes:
        print(f"\\n❌ DÉTAIL REJETS ({min(len(fichiers_rejetes), 10)} premiers):")
        for fichier in fichiers_rejetes[:10]:
            print(f"   • {fichier['nom']} - {fichier['raison']} [{fichier['dossier']}]")
        if len(fichiers_rejetes) > 10:
            print(f"   • ... et {len(fichiers_rejetes) - 10} autres")
    
    return {
        'success': True,
        'acceptes': fichiers_acceptes,
        'rejetes': fichiers_rejetes,
        'stats': {
            'filtrage': stats,
            'dossiers': dossiers_stats,
            'total_rejetes': len(fichiers_rejetes),
            'total_acceptes': len(fichiers_acceptes)
        }
    }

print("✅ Fonction search_and_filter_files définie")


✅ Fonction search_and_filter_files définie


In [7]:
# 📋 Fonction de copie (copiée du script MCP)
def copy_files_to_depot(
    fichiers_acceptes: List[Dict],
    fichiers_rejetes: List[Dict] = None,
    depot_path: str = None,
    copy_rejected: bool = True,
    create_sel_files: bool = True
) -> Dict[str, Any]:
    """Copie les fichiers vers le dossier Depots avec organisation"""
    
    depot_path = depot_path or config.depot_path
    fichiers_rejetes = fichiers_rejetes or []
    
    print(f"\\n📋 COPIE VERS DEPOTS...")
    print(f"📁 Destination: {depot_path}")
    print("-" * 50)
    
    # Créer dossier principal
    os.makedirs(depot_path, exist_ok=True)
    
    # Créer Sel_Files si demandé
    sel_files_path = None
    if create_sel_files:
        sel_files_path = os.path.join(depot_path, "Sel_Files")
        os.makedirs(sel_files_path, exist_ok=True)
        print(f"✅ Dossier Sel_Files créé")
    
    # Copie des fichiers acceptés
    copied_acceptes = 0
    taille_acceptes = 0
    errors_acceptes = []
    
    print(f"\\n🟢 Copie des fichiers acceptés...")
    for fichier in fichiers_acceptes:
        try:
            file_path = fichier["chemin"]
            file_name = fichier["nom"]
            
            # Destination selon configuration
            if create_sel_files:
                destination = os.path.join(sel_files_path, file_name)
            else:
                destination = os.path.join(depot_path, file_name)
            
            # Éviter les doublons
            if os.path.exists(destination):
                base, ext = os.path.splitext(file_name)
                counter = 1
                while os.path.exists(destination):
                    new_name = f"{base}_{counter}{ext}"
                    destination = os.path.join(os.path.dirname(destination), new_name)
                    counter += 1
            
            shutil.copy(file_path, destination)
            copied_acceptes += 1
            taille_acceptes += fichier["taille"]
            
        except Exception as e:
            error_msg = f"Erreur copie {file_name}: {e}"
            print(f"❌ {error_msg}")
            errors_acceptes.append(error_msg)
    
    print(f"✅ {copied_acceptes}/{len(fichiers_acceptes)} fichiers acceptés copiés")
    
    # Copie des fichiers rejetés
    copied_rejetes = 0
    taille_rejetes = 0
    errors_rejetes = []
    
    if copy_rejected and fichiers_rejetes:
        print(f"\\n🔶 Copie des fichiers rejetés...")
        
        autre_path = os.path.join(depot_path, "Autre")
        os.makedirs(autre_path, exist_ok=True)
        
        # Grouper par raison
        rejetes_par_raison = {}
        for fichier in fichiers_rejetes:
            raison = fichier['raison'].split(':')[0].split('(')[0].strip()
            raison_clean = raison.replace(' ', '_').replace('é', 'e').lower()
            
            if raison_clean not in rejetes_par_raison:
                rejetes_par_raison[raison_clean] = []
            rejetes_par_raison[raison_clean].append(fichier)
        
        for raison, fichiers in rejetes_par_raison.items():
            sous_dossier = os.path.join(autre_path, raison)
            os.makedirs(sous_dossier, exist_ok=True)
            
            for fichier in fichiers:
                try:
                    file_path = fichier["chemin"]
                    file_name = fichier["nom"]
                    destination = os.path.join(sous_dossier, file_name)
                    
                    # Éviter doublons
                    if os.path.exists(destination):
                        base, ext = os.path.splitext(file_name)
                        counter = 1
                        while os.path.exists(destination):
                            new_name = f"{base}_{counter}{ext}"
                            destination = os.path.join(sous_dossier, new_name)
                            counter += 1
                    
                    shutil.copy(file_path, destination)
                    copied_rejetes += 1
                    if 'taille' in fichier:
                        taille_rejetes += fichier['taille']
                        
                except Exception as e:
                    error_msg = f"Erreur copie rejeté {fichier['nom']}: {e}"
                    errors_rejetes.append(error_msg)
        
        print(f"✅ {copied_rejetes}/{len(fichiers_rejetes)} fichiers rejetés copiés")
    
    # Résumé
    print(f"\\n📊 RÉSUMÉ COPIE:")
    print(f"   🟢 Acceptés: {copied_acceptes} ({taille_acceptes/(1024*1024):.1f} MB)")
    if copy_rejected:
        print(f"   🔶 Rejetés: {copied_rejetes} ({taille_rejetes/(1024*1024):.1f} MB)")
    print(f"   📦 Total: {(taille_acceptes + taille_rejetes)/(1024*1024):.1f} MB")
    
    return {
        'success': True,
        'copied_acceptes': copied_acceptes,
        'copied_rejetes': copied_rejetes,
        'taille_acceptes': taille_acceptes,
        'taille_rejetes': taille_rejetes,
        'errors': errors_acceptes + errors_rejetes,
        'sel_files_created': create_sel_files,
        'depot_path': depot_path
    }

print("✅ Fonction copy_files_to_depot définie")


✅ Fonction copy_files_to_depot définie


In [8]:
# 🎯 EXÉCUTION DU MODE DÉMO (identique au notebook original)
print("🎯 MODE DÉMO : Reproduction du workflow Extract_Depot_Sh.ipynb")
print("=" * 70)

# Vérification préalable
print(f"📁 Vérification des chemins:")
print(f"   Source: {config.source_path}")
print(f"   Depot: {config.depot_path}")

if not os.path.exists(config.source_path):
    print(f"❌ ERREUR: Dossier source non trouvé!")
    print(f"💡 Créez le dossier {config.source_path} ou modifiez config.source_path")
else:
    print(f"✅ Dossier source trouvé")
    
    try:
        # PHASE 1: Recherche et filtrage (comme cellule 4 du notebook)
        print(f"\\n🔍 PHASE 1: Recherche et filtrage des fichiers...")
        resultat_recherche = search_and_filter_files(show_rejected=True)
        
        if not resultat_recherche['success']:
            print(f"❌ Erreur lors de la recherche: {resultat_recherche.get('error', 'Erreur inconnue')}")
        else:
            fichiers_acceptes = resultat_recherche['acceptes']
            fichiers_rejetes = resultat_recherche['rejetes']
            stats = resultat_recherche['stats']
            
            print(f"\\n📊 RÉSUMÉ RECHERCHE:")
            print(f"   ✅ Fichiers acceptés: {len(fichiers_acceptes)}")
            print(f"   ❌ Fichiers rejetés: {len(fichiers_rejetes)}")
            if len(fichiers_acceptes) + len(fichiers_rejetes) > 0:
                taux = len(fichiers_acceptes) / (len(fichiers_acceptes) + len(fichiers_rejetes)) * 100
                print(f"   📈 Taux d'acceptation: {taux:.1f}%")
            
            if len(fichiers_acceptes) == 0:
                print("⚠️ Aucun fichier accepté trouvé. Workflow terminé.")
            else:
                # PHASE 2: Copie vers Depots (comme cellule 7 du notebook)
                print(f"\\n📋 PHASE 2: Copie vers Depots...")
                resultat_copie = copy_files_to_depot(
                    fichiers_acceptes=fichiers_acceptes,
                    fichiers_rejetes=fichiers_rejetes,
                    copy_rejected=True,      # Comme dans le notebook
                    create_sel_files=True    # Comme dans le notebook
                )
                
                if not resultat_copie['success']:
                    print(f"❌ Erreur lors de la copie")
                else:
                    # RÉSUMÉ FINAL (comme dans le notebook)
                    print(f"\\n🎉 PROCESSUS TERMINÉ !")
                    print(f"   📁 Dossier Depots: {config.depot_path}")
                    if resultat_copie['sel_files_created']:
                        print(f"   📂 Sel_Files: {os.path.join(config.depot_path, 'Sel_Files')}  ← Fichiers acceptés")
                    if resultat_copie['copied_rejetes'] > 0:
                        print(f"   📂 Autre: {os.path.join(config.depot_path, 'Autre')}  ← Fichiers rejetés")
                        
                    print(f"\\n📊 BILAN FINAL:")
                    print(f"   🟢 {resultat_copie['copied_acceptes']} fichiers acceptés ({resultat_copie['taille_acceptes']/(1024*1024):.1f} MB)")
                    if resultat_copie['copied_rejetes'] > 0:
                        print(f"   🔶 {resultat_copie['copied_rejetes']} fichiers rejetés ({resultat_copie['taille_rejetes']/(1024*1024):.1f} MB)")
                    print(f"   📦 TOTAL: {(resultat_copie['taille_acceptes'] + resultat_copie['taille_rejetes'])/(1024*1024):.1f} MB")
                    
                    print(f"\\n✅ STRUCTURE CRÉÉE (identique au notebook):")
                    print(f"   📂 Depots/ contient {2 if resultat_copie['copied_rejetes'] > 0 else 1} dossier(s)")
                    if resultat_copie['sel_files_created']:
                        print(f"   ├── Sel_Files/ → {resultat_copie['copied_acceptes']} fichiers acceptés")
                    if resultat_copie['copied_rejetes'] > 0:
                        print(f"   └── Autre/ → {resultat_copie['copied_rejetes']} fichiers rejetés")
        
    except Exception as e:
        print(f"❌ Erreur dans le workflow: {e}")
        import traceback
        traceback.print_exc()


🎯 MODE DÉMO : Reproduction du workflow Extract_Depot_Sh.ipynb
📁 Vérification des chemins:
   Source: C:\Users\jcbouquin\AI\GenAI\company_process\ProcessEx
   Depot: C:\Users\jcbouquin\AI\GenAI\company_process\Depots
✅ Dossier source trouvé
\n🔍 PHASE 1: Recherche et filtrage des fichiers...
🔍 RECHERCHE ET FILTRAGE...
📁 Source: C:\Users\jcbouquin\AI\GenAI\company_process\ProcessEx
📏 Taille minimum: 500 KB
📅 Date limite: 12/09/2024
📋 Extensions: .ppt, .pptx
--------------------------------------------------
📂 Analyse: DEVIS B 2025
   ✅ 62 fichier(s) retenu(s)
\n📊 STATISTIQUES:
   📋 Examinés: 1109
   ❌ Rejetés - extension: 1017
   ❌ Rejetés - taille: 30
   ❌ Rejetés - date: 0
   ❌ Rejetés - accès: 0
   ✅ ACCEPTÉS: 62
\n❌ DÉTAIL REJETS (10 premiers):
   • 3250861_IQVIA pour Bailleul__Ciblage des MG et dermatologues.pdf - Extension non supportée [DEVIS B 2025]
   • Thumbs.db - Extension non supportée [DEVIS B 2025]
   • ~$3250861_IQVIA pour Bailleul__Ciblage des MG et dermatologues.pptx - Ta

## 🎉 Notebook de démo terminé !

### ✅ Ce que fait ce notebook :

- **Reproduit exactement** le comportement de votre notebook `Extract_Depot_Sh.ipynb`
- **Contourne les restrictions d'entreprise** (pas de dépendance MCP)
- **Fonctionne avec Jupyter** qui est déjà autorisé dans votre environnement

### 🚀 Pour utiliser :

1. **Exécutez toutes les cellules** dans l'ordre (Cell → Run All)
2. **Vérifiez les chemins** dans la cellule de configuration si nécessaire
3. **Les résultats** apparaîtront dans la dernière cellule

### 📁 Résultat attendu :

- Dossier `Depots/` créé avec la même structure que votre notebook original
- `Sel_Files/` avec les fichiers acceptés
- `Autre/` avec les fichiers rejetés par catégorie

### 💡 Si ça ne fonctionne pas :

- Vérifiez que le dossier `ProcessEx` existe
- Modifiez `config.source_path` dans la cellule 1 si nécessaire
- Les messages d'erreur vous guideront

### 🎯 Avantages de cette solution :

- ✅ **Aucune installation** supplémentaire requise
- ✅ **Fonctionne dans l'environnement d'entreprise**
- ✅ **Résultat identique** au notebook original
- ✅ **Contourne les restrictions** Azure/Git Bash
